In [1]:
import pandas as pd

## Anylizing Dataset

In [2]:
daily_price_data = pd.read_csv("NFLX_Stock_Historical_Data.csv")
daily_price_data.head(10)

,Date,Close/Last,Volume,Open,High,Low
0,04/25/2024,$564.80,3807101,$549.46,$566.54,$545.705
1,04/24/2024,$555.12,5355312,$574.31,$576.907,$551.30
2,04/23/2024,$577.75,4975520,$556.49,$579.21,$555.28
3,04/22/2024,$554.60,8597651,$550.66,$558.61,$542.01
4,04/19/2024,$555.04,16449700,$567.88,$579.00,$552.16
5,04/18/2024,$610.56,8468407,$612.10,$621.33,$605.435
6,04/17/2024,$613.69,3312222,$620.97,$620.97,$607.71
7,04/16/2024,$617.52,3519122,$607.50,$622.45,$607.50
8,04/15/2024,$607.15,3085394,$630.17,$630.17,$603.871
9,04/12/2024,$622.83,2959269,$628.23,$633.1199,$618.915


In [3]:
daily_price_data.dtypes

Date          object
Close/Last    object
Volume         int64
Open          object
High          object
Low           object
dtype: object

In [4]:
# Applying the correct formatting for prices

daily_price_data["Close/Last"] = daily_price_data["Close/Last"].str.slice(1)
daily_price_data["Open"] = daily_price_data["Open"].str.slice(1)
daily_price_data["High"] = daily_price_data["High"].str.slice(1)
daily_price_data["Low"] = daily_price_data["Low"].str.slice(1)

daily_price_data.head()

,Date,Close/Last,Volume,Open,High,Low
0,04/25/2024,564.80,3807101,549.46,566.54,545.705
1,04/24/2024,555.12,5355312,574.31,576.907,551.30
2,04/23/2024,577.75,4975520,556.49,579.21,555.28
3,04/22/2024,554.60,8597651,550.66,558.61,542.01
4,04/19/2024,555.04,16449700,567.88,579.00,552.16


In [5]:
# Converting Features to more useful dtypes

daily_price_data["Date"] = pd.to_datetime(daily_price_data["Date"])
daily_price_data["Close/Last"] = daily_price_data["Close/Last"].astype(float)
daily_price_data["Open"] = daily_price_data["Open"].astype(float)
daily_price_data["High"] = daily_price_data["High"].astype(float)
daily_price_data["Low"] = daily_price_data["Low"].astype(float)

daily_price_data.dtypes

Date          datetime64[ns]
Close/Last           float64
Volume                 int64
Open                 float64
High                 float64
Low                  float64
dtype: object

## Creating Weekly Intervals

In [6]:
daily_price_data["Date"]

0      2024-04-25
1      2024-04-24
2      2024-04-23
3      2024-04-22
4      2024-04-19
          ...    
1254   2019-05-02
1255   2019-05-01
1256   2019-04-30
1257   2019-04-29
1258   2019-04-26
Name: Date, Length: 1259, dtype: datetime64[ns]

In [7]:
daily_price_data.loc[(daily_price_data["Date"].dt.day == 1) & (daily_price_data["Date"].dt.month == 1)]
# To make sure that the Dataframe excludes holidays, I'm checking whether there was any data recorded 
# on New Years Day

,Date,Close/Last,Volume,Open,High,Low


The goal is to create **weekly interval** stock price data based the **daily interval** stock price Dataframe.

The Method:\
Create a new Dataframe to store the weekly interval data.\
The weekly <u>opening price</u> will be the opening price on <u>Monday</u>.\
The weekly <u>closing price</u> will be the closing price on <u>Friday</u>.

In [8]:
# Creating Weekly_Interval table

cols = ["Year", "Week_Nr", "Open", "High", "Low", "Volume", "Close/Last"]
weekly_price_data = pd.DataFrame(columns=cols)
weekly_price_data

,Year,Week_Nr,Open,High,Low,Volume,Close/Last


In [9]:
# Populating Weekly_Interval table

weekNums = daily_price_data["Date"].dt.isocalendar().week # To get the week numbers

weekly_price_data["Year"] = daily_price_data["Date"].dt.year
weekly_price_data["Week_Nr"] = weekNums
weekly_price_data["Open"] = daily_price_data["Open"]
weekly_price_data["High"] = daily_price_data["High"]
weekly_price_data["Low"] = daily_price_data["Low"]
weekly_price_data["Volume"] = daily_price_data["Volume"]
weekly_price_data["Close/Last"] = daily_price_data["Close/Last"]

weekly_price_data = weekly_price_data.sort_index(ascending=False)
weekly_price_data.head(8)

,Year,Week_Nr,Open,High,Low,Volume,Close/Last
1258,2019,17,368.35,375.1400,366.2400,5615814,374.85
1257,2019,18,373.68,374.5753,369.1150,3764911,371.83
1256,2019,18,369.56,374.5000,368.3480,3863973,370.54
1255,2019,18,374.00,385.9900,373.1746,9257284,378.81
1254,2019,18,378.00,383.5000,374.5100,5398167,379.06
1253,2019,18,381.53,385.0300,378.2717,5130301,385.03
1252,2019,19,377.69,381.3500,376.0000,5793144,378.67
1251,2019,19,377.00,379.9100,365.8100,6974924,370.46


### Data Cleaning

In [10]:
# Removing records (at start and end) that aren't enough to represent a full week
weekly_price_data = weekly_price_data.drop([0, 1, 2, 3, 1258], axis="index")
weekly_price_data

,Year,Week_Nr,Open,High,Low,Volume,Close/Last
1257,2019,18,373.68,374.5753,369.1150,3764911,371.83
1256,2019,18,369.56,374.5000,368.3480,3863973,370.54
1255,2019,18,374.00,385.9900,373.1746,9257284,378.81
1254,2019,18,378.00,383.5000,374.5100,5398167,379.06
1253,2019,18,381.53,385.0300,378.2717,5130301,385.03
...,...,...,...,...,...,...,...
8,2024,16,630.17,630.1700,603.8710,3085394,607.15
7,2024,16,607.50,622.4500,607.5000,3519122,617.52
6,2024,16,620.97,620.9700,607.7100,3312222,613.69
5,2024,16,612.10,621.3300,605.4350,8468407,610.56


In [11]:
WPD_ungrouped = weekly_price_data

def finalize_Weekly_DF(weeklyPriceDF: pd.DataFrame):
    weeklyPriceDF = weeklyPriceDF.groupby(["Year", "Week_Nr"]).agg({
        "Open":"first", 
        "High":"max", 
        "Low":"min", 
        "Volume":"sum",
        "Close/Last":"last" 
    })
    return weeklyPriceDF

weekly_price_data = finalize_Weekly_DF(weekly_price_data)
weekly_price_data

Open      High       Low    Volume  Close/Last
Year Week_Nr                                                  
2019 1        329.08  329.1850  321.0900   8030104      323.57
     18       373.68  385.9900  368.3480  27414636      385.03
     19       377.69  381.3500  352.7500  30879720      361.04
     20       352.29  364.0000  341.3900  30886755      354.45
     21       351.23  370.4600  345.4000  24338679      354.39
...              ...       ...       ...       ...         ...
2024 12       613.56  634.3617  608.0000  12769725      628.01
     13       627.90  634.3899  601.5900  10944787      607.33
     14       608.00  638.0000  605.5101  13519564      636.18
     15       636.39  639.0000  609.3400  12720545      622.83
     16       630.17  630.1700  552.1600  34834845      555.04

[261 rows x 5 columns]

In [12]:
WPD_ungrouped.to_csv("Datasets for Analysis/Weekly_Intervals_Ungrouped.csv")
weekly_price_data.to_csv("Datasets for Analysis/Weekly_Intervals_Grouped.csv")

After closer examination (using the CSV files), I found that week 1 of 2020 started near the end of 2019,\
which explains why the grouped Dataframe starts at Week_Nr 1 then imediately jumps to Week_Nr 18.

Also, after analysing the weeks around the end of each year, I found that around all of the 5 years,\
there are some weeks that consist 4 days (instead of the expected 5 business days). I'm not sure why.

My approach:

For week 1 of 2020, it consists of 4 days (2 in 2019 and 2 in 2020). Week 52 of 2019 has 4 days.\
I plan realicate the 1st 2 days week 1 (2020) to week 52 (2019), essentially making the week slightly longer.\
I'll then just keep the rest the same, making week 1 of 2020 very short (2 days).\
I'll leave the other 4-day weeks the same, assuming it doesn't greatly impact the results.

In [13]:
WPD_ungrouped.loc[
    (WPD_ungrouped["Year"] == 2019) & (WPD_ungrouped["Week_Nr"] == 1), "Week_Nr"
    ] = 52

WPD_ungrouped.loc[WPD_ungrouped["Year"] == 2019].tail(7)

,Year,Week_Nr,Open,High,Low,Volume,Close/Last
1092,2019,51,335.00,338.0000,330.6000,9923260,336.90
1091,2019,52,337.76,337.9500,331.0201,5767937,333.10
1090,2019,52,334.01,335.7000,331.6000,2019322,333.20
1089,2019,52,334.60,336.4623,332.0100,3595854,332.63
1088,2019,52,332.96,333.8200,326.0100,5037156,329.09
1087,2019,52,329.08,329.1850,322.8600,4315267,323.31
1086,2019,52,322.00,324.9160,321.0900,3714837,323.57


In [14]:
WPD_ungrouped.loc[WPD_ungrouped["Year"] == 2020].head(3)

,Year,Week_Nr,Open,High,Low,Volume,Close/Last
1085,2020,1,326.10,329.9800,324.78,4348493,329.81
1084,2020,1,326.78,329.8599,325.53,3795466,325.90
1083,2020,2,323.12,336.3600,321.20,5668577,335.83


In [15]:
weekly_price_data = finalize_Weekly_DF(WPD_ungrouped)
weekly_price_data

Open      High       Low    Volume  Close/Last
Year Week_Nr                                                  
2019 18       373.68  385.9900  368.3480  27414636      385.03
     19       377.69  381.3500  352.7500  30879720      361.04
     20       352.29  364.0000  341.3900  30886755      354.45
     21       351.23  370.4600  345.4000  24338679      354.39
     22       354.39  361.2000  342.9200  19407548      343.28
...              ...       ...       ...       ...         ...
2024 12       613.56  634.3617  608.0000  12769725      628.01
     13       627.90  634.3899  601.5900  10944787      607.33
     14       608.00  638.0000  605.5101  13519564      636.18
     15       636.39  639.0000  609.3400  12720545      622.83
     16       630.17  630.1700  552.1600  34834845      555.04

[260 rows x 5 columns]

In [16]:
weekly_price_data.to_csv("Finalized_Weekly_Prices.csv")